In [1]:
!pip install 'apache-airflow[all]'

     |████████████████████████████████| 4.7 MB 1.1 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 8.1 MB/s  eta 0:00:01
     |████████████████████████████████| 327 kB 13.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.7 MB 13.8 MB/s eta 0:00:01
     |████████████████████████████████| 59 kB 8.5 MB/s  eta 0:00:01
     |████████████████████████████████| 77 kB 5.1 MB/s  eta 0:00:01
     |████████████████████████████████| 54 kB 4.3 MB/s  eta 0:00:01
     |████████████████████████████████| 1.7 MB 17.8 MB/s eta 0:00:01
     |████████████████████████████████| 94 kB 4.7 MB/s  eta 0:00:01
     |████████████████████████████████| 74 kB 4.9 MB/s  eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 10.8 MB 59.1 MB/s eta 0:00:01    |███████████████████████▌        | 7.9 MB 59.1 MB/s eta 0:00:0100:01
     |████████████████████████████████| 106 kB 36

     |██████████

     |████████████████████████████████| 4.9 MB 45.2 MB/s eta 0:00:01�██████████████▎      | 3.9 MB 45.2 MB/s eta 0:00:01MB/s eta 0:00:01
     |████████████████████████████████| 131 kB 52.5 MB/s eta 0:00:01
     |████████████████████████████████| 61 kB 4.2 MB/s eta 0:00:011
     |████████████████████████████████| 3.8 MB 7.5 MB/s eta 0:00:01
     |████████████████████████████████| 248 kB 54.6 MB/s eta 0:00:01
     |████████████████████████████████| 119 kB 67.4 MB/s eta 0:00:01
     |████████████████████████████████| 41 kB 95 kB/s s eta 0:00:01
     |████████████████████████████████| 51 kB 219 kB/s  eta 0:00:01
     |████████████████████████████████| 171 kB 59.7 MB/s eta 0:00:01
     |████████████████████████████████| 255 kB 59.0 MB/s eta 0:00:01
     |████████████████████████████████| 6.0 MB 61.5 MB/s eta 0:00:01                          | 727 kB 61.5 MB/s eta 0:00:01MB/s eta 0:00:01     |██████████████████████████▌     | 5.0 MB 61.5 MB/s eta 0:00:01
     |███████████████████████████████

In [8]:
from datetime import timedelta
from airflow import DAG
from airflow.operators.bash_operator import BashOperator
from airflow.utils.dates import days_ago

In [13]:
default_args = {
    'owner' : 'airflow',
    'depends_on_past' : False,
    'start_date' : days_ago(2),
    'email' : ['airflow@example.com'],
    'email_on_failure': False,
    'email_on_retry' : False,
    'retries': 1,
    'retry_delay' : timedelta(minutes=5),
}

dag = DAG('tutorial2', default_args = default_args, description = 'A simple tutorial DAG', schedule_interval=timedelta(days=1),)

In [14]:
t1 = BashOperator(task_id='print_date',
                 bash_command='date',
                 dag = dag)

In [16]:
t2 = BashOperator(task_id='sleep',
                 depends_on_past=False,
                 bash_command='sleep 5',
                 retries=3,
                 dag = dag)

In [17]:
dag.doc_md = __doc__
t1.doc_md = """\
#### Task Documentation
You can document your task here..!
"""

templated_command = """
{% for in in range(5) %}
    echo "{{ ds }}"
    echo "{{ macros.ds_add(ds, 7)}}"
    echo "{{ params.my_param }}"
{% endfor %}
"""

In [18]:
t3 = BashOperator(task_id = 'templated',
                 depends_on_past=False,
                 bash_command=templated_command,
                 params={'my_param': 'Parameter I passed in'},
                 dag = dag,)

In [19]:
t1 >> [t2, t3]

[<Task(BashOperator): sleep>, <Task(BashOperator): templated>]

In [21]:
!airflow list_dags

[2020-09-15 10:58:35,012] {cli_action_loggers.py:107} WARNING - Failed to log action with (sqlite3.OperationalError) no such table: log
[SQL: INSERT INTO log (dttm, dag_id, task_id, event, execution_date, owner, extra) VALUES (?, ?, ?, ?, ?, ?, ?)]
[parameters: ('2020-09-15 10:58:34.987571', None, None, 'cli_list_dags', None, 'jovyan', '{"host_name": "94678fa2cd78", "full_command": "[\'/opt/conda/bin/airflow\', \'list_dags\']"}')]
(Background on this error at: http://sqlalche.me/e/13/e3q8)
[2020-09-15 10:58:35,012] {__init__.py:50} INFO - Using executor SequentialExecutor
[2020-09-15 10:58:35,012] {dagbag.py:417} INFO - Filling up the DagBag from /home/jovyan/airflow/dags
/opt/conda/lib/python3.8/site-packages/airflow/models/dag.py:1337: PendingDeprecationWarning: The requested task could not be added to the DAG because a task with task_id create_tag_template_field_result is already in the DAG. Starting in Airflow 2.0, trying to overwrite a task will raise an exception.
  warnings.warn

In [22]:
!airflow test tutorial print_date 2015-06-01

[2020-09-15 10:59:24,963] {cli_action_loggers.py:107} WARNING - Failed to log action with (sqlite3.OperationalError) no such table: log
[SQL: INSERT INTO log (dttm, dag_id, task_id, event, execution_date, owner, extra) VALUES (?, ?, ?, ?, ?, ?, ?)]
[parameters: ('2020-09-15 10:59:24.922615', 'tutorial', 'print_date', 'cli_test', '2015-06-01 00:00:00.000000', 'jovyan', '{"host_name": "94678fa2cd78", "full_command": "[\'/opt/conda/bin/airflow\', \'test\', \'tutorial\', \'print_date\', \'2015-06-01\']"}')]
(Background on this error at: http://sqlalche.me/e/13/e3q8)
[2020-09-15 10:59:24,964] {__init__.py:50} INFO - Using executor SequentialExecutor
[2020-09-15 10:59:24,964] {dagbag.py:417} INFO - Filling up the DagBag from /home/jovyan/airflow/dags
/opt/conda/lib/python3.8/site-packages/airflow/models/dag.py:1337: PendingDeprecationWarning: The requested task could not be added to the DAG because a task with task_id create_tag_template_field_result is already in the DAG. Starting in Airflo